In [1]:
# -*- coding: cp65001 -*-
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.xception import Xception, preprocess_input
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input,Dense, Dropout, Activation, Flatten, Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
import matplotlib.pyplot as plt
emotion = ['angry', 'disgust', 'fearful','happy','sad','surprised']
IMG_size = 314
base_model = keras.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(IMG_size, IMG_size, 3))
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)
model = Model(base_model.input, predictions)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
train_datagen = ImageDataGenerator(rescale=1./255,  
                                   validation_split=0.2,
                                   )
test_datagen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

training_set = train_datagen.flow_from_directory('emotion_middle',
                                                 target_size=(IMG_size,IMG_size),
                                                 classes=emotion,
                                                 subset='training',
                                                 batch_size = 16,
                                                 class_mode = 'categorical')



val_set = train_datagen.flow_from_directory('emotion_middle',
                                                 target_size=(IMG_size,IMG_size),
                                                 classes=emotion,
                                                 subset="validation",
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_lr=1e-6)
import time
a = time.time()
logic = model.fit_generator(training_set, epochs=20,  callbacks=[reduce_lr], validation_data = val_set,shuffle='true')

print(time.time()-a)
scores = model.evaluate_generator(val_set)
print(scores[1])#測試準確度


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Using TensorFlow backend.


AttributeError: 'tuple' object has no attribute 'layer'

In [ ]:

model.save('real_crop_emotion.h5')
